# LAB 6: Text classification with linear models

Objectives:

* Train and evaluate linear text classifiers using SGDClassifier
* Experiment with different feature extraction and training methods
* Log and evaluate experimental results using [mlflow](https://mlflow.org)

In [ ]:
import numpy as np
import pandas as pd
from cytoolz import *
from tqdm.auto import tqdm

tqdm.pandas()

### Load and preprocess data

In [ ]:
train = pd.read_parquet(
    "s3://ling583/rcv1-topics-train.parquet", storage_options={"anon": True}
)
test = pd.read_parquet(
    "s3://ling583/rcv1-topics-test.parquet", storage_options={"anon": True}
)

In [ ]:
train.head()

CCAT : CORPORATE/INDUSTRIAL  
ECAT : ECONOMICS  
GCAT : GOVERNMENT/SOCIAL  
MCAT : MARKETS

In [ ]:
train["topics"].value_counts()

In [ ]:
import spacy

nlp = spacy.load(
    "en_core_web_sm",
    exclude=["tagger", "parser", "ner", "lemmatizer", "attribute_ruler"],
)


def tokenize(text):
    doc = nlp.tokenizer(text)
    return [t.norm_ for t in doc if t.is_alpha]

In [ ]:
import multiprocessing as mp

In [ ]:
with mp.Pool() as p:
    train["tokens"] = pd.Series(p.imap(tokenize, tqdm(train["text"]), chunksize=100))
    test["tokens"] = pd.Series(p.imap(tokenize, tqdm(test["text"]), chunksize=100))

In [ ]:
train.head()

---

### Multinomial Naive Bayes

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

In [ ]:
mnb = make_pipeline(CountVectorizer(analyzer=identity), MultinomialNB())
mnb.fit(train["tokens"], train["topics"])
predicted = mnb.predict(test["tokens"])
print(classification_report(test["topics"], predicted))

In [ ]:
import logger
import mlflow
from logger import log_search, log_test

In [ ]:
mlflow.set_experiment("lab-6")
log_test(mnb, test["topics"], predicted)

---

### Hyperparameters

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:44627")
client

In [ ]:
from dask_ml.model_selection import RandomizedSearchCV
from scipy.stats.distributions import loguniform, randint, uniform

In [ ]:
from warnings import simplefilter

simplefilter(action="ignore", category=FutureWarning)

In [ ]:
mlflow.set_experiment("lab-6/MultinomialNB")

In [ ]:
%%time

search = RandomizedSearchCV(
    mnb,
    {
        "countvectorizer__min_df": randint(1, 10),
        "countvectorizer__max_df": uniform(0.5, 0.5),
        "multinomialnb__alpha": loguniform(1e-8, 100.0),
    },
    n_iter=25,
    scoring="f1_macro",
)
search.fit(train["tokens"], train["topics"])
log_search(search)

In [ ]:
%%time

search = RandomizedSearchCV(
    mnb,
    {
        "countvectorizer__min_df": randint(1, 10),
        "countvectorizer__max_df": uniform(0.5, 0.5),
        "multinomialnb__alpha": loguniform(1e-3, 1.0),
    },
    n_iter=25,
    scoring="f1_macro",
)
search.fit(train["tokens"], train["topics"])
log_search(search)

In [ ]:
%%time

search = RandomizedSearchCV(
    mnb,
    {
        "countvectorizer__min_df": randint(1, 10),
        "countvectorizer__max_df": uniform(0.5, 0.5),
        "multinomialnb__alpha": [0.1],
    },
    n_iter=25,
    scoring="f1_macro",
)
search.fit(train["tokens"], train["topics"])
log_search(search)

----

### Optimized model

In [ ]:
mnb = make_pipeline(
    CountVectorizer(analyzer=identity, min_df=2, max_df=0.7), MultinomialNB(alpha=0.1)
)
mnb.fit(train["tokens"], train["topics"])
predicted = mnb.predict(test["tokens"])
print(classification_report(test["topics"], predicted))

In [ ]:
mlflow.set_experiment("lab-6")
log_test(mnb, test["topics"], predicted)